<a href="https://colab.research.google.com/github/distractedm1nd/mar2moon/blob/main/Wav2VecFirstSteps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
#%%capture
#!pip install git+https://github.com/huggingface/datasets.git
#!pip install git+https://github.com/huggingface/transformers.git
#!pip install soundfile
#!pip install pydub
#!pip install jiwer

In [5]:
#!sudo curl -L https://yt-dl.org/downloads/latest/youtube-dl -o /usr/local/bin/youtube-dl

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100     3  100     3    0     0      2      0  0:00:01  0:00:01 --:--:--     2
100     3  100     3    0     0      2      0  0:00:01  0:00:01 --:--:--     2
100   614  100   614    0     0    390      0  0:00:01  0:00:01 --:--:--   390
100 1797k  100 1797k    0     0   962k      0  0:00:01  0:00:01 --:--:-- 13.2M


In [6]:
#!sudo chmod a+rx /usr/local/bin/youtube-dl

# Downloading Video and Audio
Here I download the audio as `wav` and download the automatic transcription by google in the `srt` format.

In [1]:
!youtube-dl -x --audio-format wav --write-auto-sub --convert-subs=srt https://www.youtube.com/watch\?v\=zbtC2-caXaE

[youtube] zbtC2-caXaE: Downloading webpage
[info] Writing video subtitles to: Uniswap on Arbitrum, ETH vs BTC and more - The Daily Gwei Refuel #137 - Ethereum Updates-zbtC2-caXaE.en.vtt
[download] Destination: Uniswap on Arbitrum, ETH vs BTC and more - The Daily Gwei Refuel #137 - Ethereum Updates-zbtC2-caXaE.webm
[download] 100% of 30.57MiB in 00:04
[ffmpeg] Destination: Uniswap on Arbitrum, ETH vs BTC and more - The Daily Gwei Refuel #137 - Ethereum Updates-zbtC2-caXaE.wav
Deleting original file Uniswap on Arbitrum, ETH vs BTC and more - The Daily Gwei Refuel #137 - Ethereum Updates-zbtC2-caXaE.webm (pass -k to keep)
[ffmpeg] Converting subtitles
Deleting original file Uniswap on Arbitrum, ETH vs BTC and more - The Daily Gwei Refuel #137 - Ethereum Updates-zbtC2-caXaE.en.vtt (pass -k to keep)


In [2]:
!mv 'Uniswap on Arbitrum, ETH vs BTC and more - The Daily Gwei Refuel #137 - Ethereum Updates-zbtC2-caXaE.en.srt' 'subtitles.srt'
!mv 'Uniswap on Arbitrum, ETH vs BTC and more - The Daily Gwei Refuel #137 - Ethereum Updates-zbtC2-caXaE.wav' '07-06-2021-TheDailyGwei_RAW.wav'

The original `wav` file is stereo and has like a 40k sampling rate so here I scale it down to 16k and mono

In [3]:
from pydub import AudioSegment
sound = AudioSegment.from_wav("07-06-2021-TheDailyGwei_RAW.wav")
sound = sound.set_channels(1)
sound = sound.set_frame_rate(16000)
sound.export("07-06-2021-TheDailyGwei.wav", format="wav")

<_io.BufferedRandom name='07-06-2021-TheDailyGwei.wav'>

Here is how the normal srt file looks.

In [4]:
!head subtitles.srt

1
00:00:00,160 --> 00:00:01,510

hello everyone and welcome to another

2
00:00:01,510 --> 00:00:01,520

 



Extracting ONLY the text from the srt format

In [5]:
import re
import sys
 
subs_file = "subtitles.srt"
out_file = "subtitles_as_text.txt"
bad_words = ['-->','</c>']
 
contains_badwords = lambda line : any(bad_word in line for bad_word in bad_words)
 
with open(subs_file) as oldfile, open(out_file, 'w') as newfile:
    lines = []
    for line in oldfile:
        if not contains_badwords(line) and not line in lines and not line.strip().isdecimal():
            lines.append(line)
    newfile.writelines(lines)

In [6]:
!head subtitles_as_text.txt


hello everyone and welcome to another
 
episode of the daily gray refuel where i
recap the latest news in the ethereum
ecosystem
i'm your host anthony sosano and today's
the 7th of june 2021
all right everyone let's get into it uh
so before we go over the news from the


I cant feed in the entire audio file at once to the model (too long, takes too much memory) so I split it up with ffmpeg

In [7]:
!mkdir audio
!mkdir audio_by_minute
!ffmpeg -i 07-06-2021-TheDailyGwei.wav -f segment -segment_time 10 -c copy audio/out%03d.wav
!ffmpeg -i 07-06-2021-TheDailyGwei.wav -f segment -segment_time 60 -c copy audio_by_minute/out%03d.wav

ffmpeg version 4.3.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with Apple clang version 12.0.0 (clang-1200.0.32.27)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/4.3.1_4 --enable-shared --enable-pthreads --enable-version3 --enable-avresample --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-librtmp --enable-libspeex --enable-libsoxr --enable-videotoolbox --disable-libjack --disable-indev=jack
  libavutil      56. 51.100 /

Let's see if the audio loads right...

In [2]:
import IPython.display as ipd
import numpy as np
import soundfile as sf
audio, sampling_rate = sf.read("audio/out021.wav")
print(audio, sampling_rate)
ipd.Audio(data=np.asarray(audio), autoplay=True, rate=sampling_rate)


[ 0.01159668 -0.00186157 -0.0324707  ... -0.01074219 -0.01245117
 -0.01376343] 16000


Okay. Now we can load one of the pretrained wav2vec models from Facebook. We will fine tune this later with our own labels

In [3]:
from transformers import Wav2Vec2Tokenizer, Wav2Vec2ForCTC
from datasets import load_dataset
import torch

# load model and tokenizer
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-large-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h")

/usr/local/lib/python3.9/site-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:417: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  warnings.warn(
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
Lets feed one audio file into the model to see what comes out

SyntaxError: ignored

In [13]:
input_values = tokenizer(audio, return_tensors="pt", padding="longest").input_values  # Batch size 1


# retrieve logits
logits = model(input_values).logits

# take argmax and decode
predicted_ids = torch.argmax(logits, dim=-1)
tokenizer.batch_decode(predicted_ids)

["NG IS QUITE RIGHT NOW THE ONLY THING THAT'S REALLY GOING UP CONTINUOUSLY IS LIKE THE ETH BITI C RATIO I EVERYTHING ELSE IS REALLY QUITE LIKE DFIIN GENERAL'S VERY QUIET AH NER BITQUAIN LOOKS"]

And here I just increase the batch size by loading multiple audio files. Can't really do much in colab because it has low memory

In [ ]:
# Colab runs out of memory when doing this but you can run it on your local machine to see the results
for i in range(10):
  audio = [sf.read(f"audio/out{iter:03d}.wav")[0] for iter in range(i * 5, (i * 5) + 5)]
  input_values = tokenizer(audio, return_tensors="pt", padding="longest").input_values
  
  # retrieve logits
  logits = model(input_values).logits
  
  # take argmax and decode
  predicted_ids = torch.argmax(logits, dim=-1)
  tokenizer.batch_decode(predicted_ids)